In [1]:
!pip install pygame
import pygame
print(pygame.__version__)

pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html
2.6.1


In [3]:
import pygame
import random
import time  

WIDTH, HEIGHT = 800, 600
FPS = 15  # تقليل FPS لجعل الحركة أبطأ
CELL_SIZE = 40

# ألوان
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GREEN = (0, 255, 0)
RED = (255, 0, 0)
EXIT_COLOR = (0, 0, 255)                   
PURPLE = (128, 0, 128)  

# إنشاء نافذة اللعبة
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Maze Game")
clock = pygame.time.Clock()

# إنشاء المتاهة
def generate_maze(width, height):
    maze = [[1 for _ in range(width)] for _ in range(height)]
    stack = [(1, 1)]
    maze[1][1] = 0

    while stack:
        x, y = stack[-1]
        neighbors = []

        for dx, dy in [(-2, 0), (2, 0), (0, -2), (0, 2)]:
            nx, ny = x + dx, y + dy
            if 0 < nx < width and 0 < ny < height and maze[ny][nx] == 1:
                neighbors.append((nx, ny))

        if neighbors:
            nx, ny = random.choice(neighbors)
            maze[(y + ny) // 2][(x + nx) // 2] = 0
            maze[ny][nx] = 0
            stack.append((nx, ny))
        else:
            stack.pop()

    return maze

# رسم المتاهة
def draw_maze(maze, exit_x, exit_y):
    for y, row in enumerate(maze):
        for x, cell in enumerate(row):
            color = BLACK if cell else WHITE
            pygame.draw.rect(screen, color, (x * CELL_SIZE, y * CELL_SIZE, CELL_SIZE, CELL_SIZE))

    # رسم نقطة الخروج
    pygame.draw.rect(screen, EXIT_COLOR, (exit_x * CELL_SIZE, exit_y * CELL_SIZE, CELL_SIZE, CELL_SIZE))

# اللعبة الرئيسية
def main():
    game_won = False
    player_x, player_y = 1, 1
    start_time = time.time()  # بداية الوقت عند بداية اللعبة
    elapsed_time = 0  # وقت اللعبة من بداية التوقيت

    while True:
        # إنشاء المتاهة مرة أخرى إذا كانت اللعبة قد انتهت
        maze = generate_maze(WIDTH // CELL_SIZE, HEIGHT // CELL_SIZE)
        
        # تحديد نقطة الخروج (أسفل يمين المتاهة)
        exit_x, exit_y = len(maze[0]) - 2, len(maze) - 2

        running = True
        while running:
            screen.fill(GREEN)
            draw_maze(maze, exit_x, exit_y)

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    return

            # حساب الوقت المنقضي فقط إذا كانت اللعبة لم تنتهى
            if not game_won:
                elapsed_time = time.time() - start_time  # حساب الوقت منذ بداية اللعبة
                remaining_time = 15
            - int(elapsed_time)  # الوقت المتبقي
                if remaining_time <= 0:  # إذا انتهى الوقت
                    game_won = False
                    font = pygame.font.SysFont(None, 55)
                    text = font.render("You Lose!", True, (255, 0, 0))  # عرض "خسرت"
                    screen.blit(text, (WIDTH // 3, HEIGHT // 3))

                    # عرض تعليمات لإعادة اللعب أو الخروج
                    font = pygame.font.SysFont(None, 40)
                    text = font.render("Press 'P' to play again or 'Q' to quit.", True, PURPLE)
                    screen.blit(text, (WIDTH // 3 - 50, HEIGHT // 2 + 50))

                    pygame.display.flip()
                    clock.tick(FPS)
                    while True:  
                        for event in pygame.event.get():
                            if event.type == pygame.QUIT:
                                pygame.quit()
                                return
                            if event.type == pygame.KEYDOWN:
                                if event.key == pygame.K_q:
                                    pygame.quit()
                                    return
                                if event.key == pygame.K_p:
                                    game_won = False
                                    player_x, player_y = 1, 1  # إعادة تموضع اللاعب
                                    start_time = time.time()  # إعادة تعيين الوقت عند إعادة اللعبة
                                    break  # الخروج من حلقة الانتظار

                # عرض الوقت المتبقي
                font = pygame.font.SysFont(None, 40)
                text = font.render(f"Time: {remaining_time}s", True, (255, 255, 255))
                screen.blit(text, (WIDTH - 150, 20))

            if not game_won:
                keys = pygame.key.get_pressed()

                # التحقق من الحدود قبل التحرك
                if keys[pygame.K_UP] and player_y > 0 and maze[player_y - 1][player_x] == 0:
                    player_y -= 1
                if keys[pygame.K_DOWN] and player_y < len(maze) - 1 and maze[player_y + 1][player_x] == 0:
                    player_y += 1
                if keys[pygame.K_LEFT] and player_x > 0 and maze[player_y][player_x - 1] == 0:
                    player_x -= 1
                if keys[pygame.K_RIGHT] and player_x < len(maze[0]) - 1 and maze[player_y][player_x + 1] == 0:
                    player_x += 1

            # رسم اللاعب
            pygame.draw.rect(screen, RED, (player_x * CELL_SIZE, player_y * CELL_SIZE, CELL_SIZE, CELL_SIZE))

            # التحقق إذا وصل اللاعب إلى نقطة الخروج
            if player_x == exit_x and player_y == exit_y:
                game_won = True  # تغير الحالة عندما يفوز اللاعب
                elapsed_time = time.time() - start_time       
                font = pygame.font.SysFont(None, 55)
                text = font.render("You Win!", True, (0, 255, 0))
                screen.blit(text, (WIDTH // 3, HEIGHT // 3))

                # عرض الوقت الذي استغرقه اللاعب
                font = pygame.font.SysFont(None, 40)
                text = font.render(f"Time: {int(elapsed_time)}s", True, (255, 255, 255))
                screen.blit(text, (WIDTH // 3 - 50, HEIGHT // 2))

                # عرض تعليمات لإعادة اللعب أو الخروج
                font = pygame.font.SysFont(None, 40)
                text = font.render("Press 'P' to play again or 'Q' to quit.", True, PURPLE)
                screen.blit(text, (WIDTH // 3 - 50, HEIGHT // 2 + 50))

            pygame.display.flip()
            clock.tick(FPS)  # تقليل FPS لجعل الحركة أبطأ

            # التحقق من المفاتيح بعد الفوز أو الخسارة
            if game_won or remaining_time <= 0:
                keys = pygame.key.get_pressed()
                if keys[pygame.K_q]:
                    pygame.quit()
                    return
                if keys[pygame.K_p]:
                    game_won = False
                    player_x, player_y = 1, 1  
                    start_time = time.time()  # إعادة تعيين الوقت عند إعادة اللعبة

if __name__ == "__main__":
    main()

IndentationError: unexpected indent (1725213882.py, line 87)